In [ ]:
# add autoreload
%load_ext autoreload
%autoreload 2
import sickle
from tqdm import tqdm
import os
import requests
import lxml
import bs4
import random
from time import sleep


In [70]:
tlist = ['athero', 'plaque', 'cardiovascular', 'cardiogram', 'cardiology', 'heart', 'vascular', 'angiogram', 
         'cardiologie', 'hartziekte', 'vaatziekte', 'medical', 'clinical', 'surgical', 'metabolic', 'myocard']
institute = 'VU'
base_url =   'https://research.vu.nl/ws/oai' #'https://repository.ubn.ru.nl/oai/openaire'  https://scholarlypublications.universiteitleiden.nl/oai2, http://dspace.library.uu.nl/oai/dissertation
pdf_path = '//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PhDTheses'

In [ ]:
OpenAIRE_institutes = ['VU']

In [ ]:
sickler = sickle.Sickle(base_url)


In [ ]:
sets = sickler.ListSets()

In [ ]:
Sets = {}
for s in sets:
    Sets[s.setSpec]  = s.setName    

In [ ]:
keywords = ['medisch', 'medical', 'diss', 'phd', 'thesis']
if institute in OpenAIRE_institutes:
    keywords = keywords + ['publications:withfiles']

Sets_to_mine = []
for key, val in Sets.items():
    if any([c in val.lower() for c in keywords]) | any([c in key.lower() for c in keywords]):
        print(f'Set: {key} contains keyword')
        Sets_to_mine.append(key)

In [ ]:
# get records 
from collections import defaultdict
records_lists = defaultdict(list)
for set_to_mine in tqdm(Sets_to_mine):
    print(f"Mining from set: {set_to_mine}")
    records = sickler.ListRecords(metadataPrefix='oai_dc', 
                                  ignore_deleted=True, 
                                  set=set_to_mine) # dissertation com_1874_298213
    for record in records:
        records_lists[set_to_mine].append(record)

In [71]:
filtered_records_lists = defaultdict(list)
for set_to_mine in Sets_to_mine:
    relevant_counter = 0
    for r in tqdm(records_lists[set_to_mine]):
        meta = r.get_metadata()
        relevant = False
        
        try:
            if any([t in subj for subj in meta['subject'] for t in tlist]):
                relevant = True
        except:
            pass

        try:
            if any([t in subj for subj in meta['title'] for t in tlist]):
                relevant = True
        except:
            pass

        try:
            if any([t in subj for subj in meta['description'] for t in tlist]):
                relevant = True
        except:
            pass 

        try:
            if ('pdf' in meta['format'][0].lower()) & relevant:
                relevant = True
            else:
                continue
        except:
            pass
        
        try:
            if ('doctoral' in meta['type'][0].lower()) & relevant:
                relevant = True
            else:
                relevant = False
        except:
            pass

        try:
            if ('embargo' in meta['rights'][0].lower()) & relevant:
                relevant = False
        except:
            pass  
        
        if relevant:
            relevant_counter += 1
            filtered_records_lists[set_to_mine].append(r)

    print(f'Found {relevant_counter} relevant records in set: {set_to_mine}')      

100%|██████████| 32671/32671 [00:02<00:00, 12784.54it/s]

Found 657 relevant records in set: publications:withFiles


In [108]:
link_list = []
error_list = []

for set_to_mine in filtered_records_lists.keys():
    for record in tqdm(filtered_records_lists[set_to_mine]):
        META = record.get_metadata()

        try:
            List_of_identifiers = META['identifier'] if 'identifier' in META.keys() else ['']
            Title = META['title'] if 'title' in META.keys() else ['']
            Description = META['description'] if 'description' in META.keys() else ['']
            Date = META['date'] if 'date' in META.keys() else ['']
            Language = META['language'] if 'language' in META.keys() else ['']       
            Creator = META['creator'] if 'creator' in META.keys() else ['']
            link_list.append({'Set': set_to_mine, 
                            'Link': List_of_identifiers, 
                            'Title': Title,
                            'Description': Description,
                            'Date': Date ,
                            'Language': Language,
                            'Creator': Creator
                            }
                            )
        except Exception as e:
            error_list.append(f"Error parsing {e}: {META}: ")
            pass

100%|██████████| 657/657 [00:00<00:00, 12514.40it/s]


In [ ]:

            if institute == 'LUMC':
                try:
                    link = meta['identifier'][-1] 
                    # identify first url in list
                    found_link = False
                    for l in meta['identifier']:
                        if ('http:' in l) or ('https:' in l):
                            link = l
                            found_link = True
                            break
                    if not found_link:
                        error_list.append(f'No link found for {meta["identifier"]}')  

                    doc_id = link.split('/')[-1]
                    doc_id_int = int(doc_id)+2

                    link = f"https://scholarlypublications.universiteitleiden.nl/handle/1887/{doc_id}"
                    linkPdf = f"https://scholarlypublications.universiteitleiden.nl/access/item%3A{doc_id_int}/download"

                    # extract through link url. The directory can be found in <li class='ubl-file-download'> <a href='...'>
                    # only if <a href in ubl-file-view is "full"
                    r = requests.get(link)
                    # random number between 0.5 and 2.5 seconds
                    rndSleep = round(random.uniform(0.5, 2.5), 2)
                    sleep(rndSleep)
                    soup = bs4.BeautifulSoup(r.text, 'html.parser')
                    found, dsfound, esfound = False, False, False
                    for _res in soup.findAll('li', {'class':'ubl-file-view'}):
                        if _res.a is not None:
                            if _res.a.contents[0].strip().lower() == 'full text':
                                _pdfdir = _res.a['href']
                                found = True
                            elif _res.a.contents[0].strip().lower() == 'summary in dutch':
                                _dutch_summary = _res.a['href']
                                dsfound = True
                            elif _res.a.contents[0].strip().lower() == 'summary in english':
                                _english_summary = _res.a['href']
                                esfound = True

                    linkPdfAlt = f"https://scholarlypublications.universiteitleiden.nl{_pdfdir}" if found else None
                    DutchSummaryLink = f"https://scholarlypublications.universiteitleiden.nl{_dutch_summary}" if dsfound else None
                    EnglishSummaryLink = f"https://scholarlypublications.universiteitleiden.nl{_english_summary}" if esfound else None
                    
                    try:
                        lang = meta['language'][0]
                    except:
                        lang = None
                except Exception as e:
                    error_list.append(f'Error: {e} for link: {link}, with meta data: {meta["identifier"]}')
                    pass
                    #raise ValueError(f'Could not find pdf link for {link}, with error raised: {e}')
            elif institute == 'UU':                
                link = meta['identifier'][0]
                baselink = link.replace('dspace.library.uu.nl/', 'dspace.library.uu.nl/bitstream/')
                linkPdf = baselink + '/full.pdf'

                try:
                    linkPdfAlt = baselink +'/'+meta['creator'][0].split(',')[0].lower()+'.pdf'
                except:
                    linkPdfAlt = None
                
                DutchSummaryLink = None
                EnglishSummaryLink = None
                try:
                    lang = meta['language'][0]
                except:
                    lang = None

            elif institute == 'Radboud':
                try:
                    found_link = False
                    found_pdf_link = False
                    for l in meta['identifier']:
                        if l is not None:
                            if (('http:' in l) or ('https:' in l)) & (l.split("/")[-1].endswith('.pdf')):
                                linkPdf = l
                                found_pdf_link = True                            
                            elif (('http:' in l) or ('https:' in l)):
                                link = l
                                found_link = True                           

                    if not found_link:                        
                        error_list.append(f'No link found for {meta["identifier"]}')  
                        continue
                    if not found_pdf_link:
                        error_list.append(f'No pdf link found for {meta["identifier"]}')
                        linkPdf = None

                    linkPdfAlt = None
                    DutchSummaryLink = None
                    EnglishSummaryLink = None
                    try:
                        lang = meta['language'][0]
                    except:
                        lang = None
                except Exception as e:
                    error_list.append(f'Error: {e} for link: {link}, with meta data: {meta["identifier"]}')
                    continue
                    #raise ValueError(f'Could not find pdf link for {link}, with error raised: {e}')                


In [107]:
def extract_pdf_links(links, institute=None):
    pdf_links = []
    inclusion_terms = ['abstract', 'full', 'complete', 'samenvatting', 'summary', 'thesis']
    if institute in ['VU']:
        for link in links:
            if (link.lower().startswith('http')) & \
                    (link.lower().endswith('.pdf')):
                if any([t in link.lower() for t in inclusion_terms]):
                    pdf_links.append(link)
    elif institute in ['Radboud']:
        

    return pdf_links

In [120]:
link_pdf_list = []
for l in link_list:
    links = extract_pdf_links(l['Link'], institute=institute)
    tmp = []
    for _l in links:
        pdfPath = _l.split("/")[-1].replace("%20", "_").rstrip('.pdf')
        pdfPath = institute + "_" + l['Creator'][0] + "_" + l['Date'][0] + "_" + pdfPath
        pdfPath = pdfPath.replace(",", "")
        pdfPath = pdfPath.replace(":", "")
        pdfPath = pdfPath.replace(".", "")
        pdfPath = pdfPath.replace(" ", "")
        pdfPath = os.path.join(pdf_path, pdfPath+".pdf")
        tmp.append((_l, pdfPath))
    l['pdf_links'] = tmp
    link_pdf_list.append(l)

In [126]:
# remove duplicate entries, based on the title
title_set = set()
unique_link_list = []
for el in link_pdf_list:
    try:
        title = el['Title'][0]
        if title not in title_set:
            unique_link_list.append(el)
            title_set.add(title)
    except Exception as e:
        pass

In [ ]:
# now we parse the link list and download the pdfs
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.107 Safari/537.36' }

# add sleep

def pdf_writer(pdf_url, _pdf_path):
    r = requests.get(pdf_url, stream=True)
    if r.status_code == 200:
        with open(_pdf_path, 'wb') as f:
            for i,chunk in enumerate(r.iter_content(chunk_size=1024)): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        return True, 200
    else:
        return False, r.status_code

pdf_error_list = []
skipped_list = []
src_list = []
rcode_list = []
extract_full_text = True
for link in tqdm(unique_link_list):
    for lt in link['pdf_links']:
        _pdf_path = lt[1]
        pdf_url = lt[0]
        return_code = None

        if os.path.isfile(_pdf_path):
            skipped_list.append(f'Pdf already exists: {_pdf_path}')
        else:
            # try to download the pdf
            try:
                write_file, return_code = pdf_writer(pdf_url, _pdf_path)
            except Exception as e:
                pdf_error_list.append(f'Error: {e} for link: {link["Link"]}')  

            if return_code == 429:
                print(f'Rate limit exceeded...sleeping 30 seconds')
                sleep(30)
                continue
            else:
                sleep(4)  
    
print(f'Found {len(pdf_error_list)} errors while downloading pdfs')
print(f'Skipped {len(skipped_list)} pdfs because they already existed')